# Import

In [162]:
#packages import
!pip install kaggle
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import os
os.environ['KAGGLE_USERNAME'] = "sabrinaauger"
os.environ['KAGGLE_KEY'] = "8db79b4ccae98a2c74af940e1f1b7c35"
import kaggle

# Import the dataset from Kaggle

In [163]:
#Retrieve the data from Kaggle platform
!kaggle datasets download -d zynicide/wine-reviews

wine-reviews.zip: Skipping, found more recently modified local copy (use --force to force download)


In [164]:
#Unzip the data zip file
!unzip wine-reviews.zip

Archive:  wine-reviews.zip
replace winemag-data-130k-v2.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: ^C


In [175]:
#Retrieve the data from csv unzipped files into two dataframes
wine1_df=pd.read_csv("/Users/sabrinaauger/code/sabrinaauger/wino/raw_data/winemag-data-130k-v2.csv")

In [166]:
wine2_df=pd.read_csv("/Users/sabrinaauger/code/sabrinaauger/wino/raw_data/winemag-data_first150k.csv")

In [173]:
wine2_df.head()

,Unnamed: 0,country,description,designation,points,price,province,region_1,region_2,variety,winery
0,0,US,This tremendous 100% varietal wine hails from ...,Martha's Vineyard,96,235.0,California,Napa Valley,Napa,Cabernet Sauvignon,Heitz
1,1,Spain,"Ripe aromas of fig, blackberry and cassis are ...",Carodorum Selección Especial Reserva,96,110.0,Northern Spain,Toro,NaN,Tinta de Toro,Bodega Carmen Rodríguez
2,2,US,Mac Watson honors the memory of a wine once ma...,Special Selected Late Harvest,96,90.0,California,Knights Valley,Sonoma,Sauvignon Blanc,Macauley
3,3,US,"This spent 20 months in 30% new French oak, an...",Reserve,96,65.0,Oregon,Willamette Valley,Willamette Valley,Pinot Noir,Ponzi
4,4,France,"This is the top wine from La Bégude, named aft...",La Brûlade,95,66.0,Provence,Bandol,NaN,Provence red blend,Domaine de la Bégude


In [174]:
wine2_df.head()

,Unnamed: 0,country,description,designation,points,price,province,region_1,region_2,variety,winery
0,0,US,This tremendous 100% varietal wine hails from ...,Martha's Vineyard,96,235.0,California,Napa Valley,Napa,Cabernet Sauvignon,Heitz
1,1,Spain,"Ripe aromas of fig, blackberry and cassis are ...",Carodorum Selección Especial Reserva,96,110.0,Northern Spain,Toro,NaN,Tinta de Toro,Bodega Carmen Rodríguez
2,2,US,Mac Watson honors the memory of a wine once ma...,Special Selected Late Harvest,96,90.0,California,Knights Valley,Sonoma,Sauvignon Blanc,Macauley
3,3,US,"This spent 20 months in 30% new French oak, an...",Reserve,96,65.0,Oregon,Willamette Valley,Willamette Valley,Pinot Noir,Ponzi
4,4,France,"This is the top wine from La Bégude, named aft...",La Brûlade,95,66.0,Provence,Bandol,NaN,Provence red blend,Domaine de la Bégude


# Data vizualisation

In [169]:
#Columns overview of both datasets
wine1_df.columns, wine2_df.columns

(Index(['Unnamed: 0', 'country', 'description', 'designation', 'points',
        'price', 'province', 'region_1', 'region_2', 'taster_name',
        'taster_twitter_handle', 'title', 'variety', 'winery'],
       dtype='object'),
 Index(['Unnamed: 0', 'country', 'description', 'designation', 'points',
        'price', 'province', 'region_1', 'region_2', 'variety', 'winery'],
       dtype='object'))

In [185]:
#Analysing the variety column and the unique value
wine1_df['variety'].unique()

array(['White Blend', 'Portuguese Red', 'Pinot Gris', 'Riesling',
       'Pinot Noir', 'Tempranillo-Merlot', 'Frappato', 'Gewürztraminer',
       'Cabernet Sauvignon', 'Nerello Mascalese', 'Chardonnay', 'Malbec',
       'Tempranillo Blend', 'Meritage', 'Red Blend', 'Merlot',
       "Nero d'Avola", 'Chenin Blanc', 'Gamay', 'Sauvignon Blanc',
       'Viognier-Chardonnay', 'Primitivo', 'Catarratto', 'Inzolia',
       'Petit Verdot', 'Monica', 'Bordeaux-style White Blend', 'Grillo',
       'Sangiovese', 'Cabernet Franc', 'Champagne Blend',
       'Bordeaux-style Red Blend', 'Aglianico', 'Petite Sirah',
       'Touriga Nacional', 'Carmenère', 'Albariño', 'Petit Manseng',
       'Rosé', 'Zinfandel', 'Vernaccia', 'Rosato', 'Grüner Veltliner',
       'Viognier', 'Vermentino', 'Grenache Blanc', 'Syrah', 'Nebbiolo',
       'Shiraz-Cabernet Sauvignon', 'Pinot Blanc', 'Alsace white blend',
       'Barbera', 'Rhône-style Red Blend', 'Portuguese White', 'Graciano',
       'Tannat-Cabernet', 'Sauvign

In [176]:
# Looking at the description of 1 specific wine
index_of_record_to_view = 3
des = wine2_df.loc[index_of_record_to_view, 'description']
des

"This spent 20 months in 30% new French oak, and incorporates fruit from Ponzi's Aurora, Abetina and Madrona vineyards, among others. Aromatic, dense and toasty, it deftly blends aromas and flavors of toast, cigar box, blackberry, black cherry, coffee and graphite. Tannins are polished to a fine sheen, and frame a finish loaded with dark chocolate and espresso. Drink now through 2032."

# Preprocessing the data 

## Cleaning wine1_df dataset 

### Removing Unuseful columns for the recommendation system

In [177]:
#Remove both columns taster name and taster twitter_handle as they are not relevant for our analysis
wine_review1_df=wine1_df.drop(columns=["taster_twitter_handle","taster_name"])
wine_review1_df.head()


,Unnamed: 0,country,description,designation,points,price,province,region_1,region_2,title,variety,winery
0,0,Italy,"Aromas include tropical fruit, broom, brimston...",Vulkà Bianco,87,NaN,Sicily & Sardinia,Etna,NaN,Nicosia 2013 Vulkà Bianco (Etna),White Blend,Nicosia
1,1,Portugal,"This is ripe and fruity, a wine that is smooth...",Avidagos,87,15.0,Douro,NaN,NaN,Quinta dos Avidagos 2011 Avidagos Red (Douro),Portuguese Red,Quinta dos Avidagos
2,2,US,"Tart and snappy, the flavors of lime flesh and...",NaN,87,14.0,Oregon,Willamette Valley,Willamette Valley,Rainstorm 2013 Pinot Gris (Willamette Valley),Pinot Gris,Rainstorm
3,3,US,"Pineapple rind, lemon pith and orange blossom ...",Reserve Late Harvest,87,13.0,Michigan,Lake Michigan Shore,NaN,St. Julian 2013 Reserve Late Harvest Riesling ...,Riesling,St. Julian
4,4,US,"Much like the regular bottling from 2012, this...",Vintner's Reserve Wild Child Block,87,65.0,Oregon,Willamette Valley,Willamette Valley,Sweet Cheeks 2012 Vintner's Reserve Wild Child...,Pinot Noir,Sweet Cheeks


In [141]:
#Extracting the year from the column title

# Extract the year from title into a dedicated column using regular expression
#regular instruction (\d{4}) simply looks for four consecutive digits in the strings
wine_review1_df['Year']=wine_review1_df['title'].str.extract(r'(\d{4})')
wine_review1_df.head()

,Unnamed: 0,country,description,designation,points,price,province,region_1,region_2,title,variety,winery,Year
0,0,Italy,"Aromas include tropical fruit, broom, brimston...",Vulkà Bianco,87,NaN,Sicily & Sardinia,Etna,NaN,Nicosia 2013 Vulkà Bianco (Etna),White Blend,Nicosia,2013
1,1,Portugal,"This is ripe and fruity, a wine that is smooth...",Avidagos,87,15.0,Douro,NaN,NaN,Quinta dos Avidagos 2011 Avidagos Red (Douro),Portuguese Red,Quinta dos Avidagos,2011
2,2,US,"Tart and snappy, the flavors of lime flesh and...",NaN,87,14.0,Oregon,Willamette Valley,Willamette Valley,Rainstorm 2013 Pinot Gris (Willamette Valley),Pinot Gris,Rainstorm,2013
3,3,US,"Pineapple rind, lemon pith and orange blossom ...",Reserve Late Harvest,87,13.0,Michigan,Lake Michigan Shore,NaN,St. Julian 2013 Reserve Late Harvest Riesling ...,Riesling,St. Julian,2013
4,4,US,"Much like the regular bottling from 2012, this...",Vintner's Reserve Wild Child Block,87,65.0,Oregon,Willamette Valley,Willamette Valley,Sweet Cheeks 2012 Vintner's Reserve Wild Child...,Pinot Noir,Sweet Cheeks,2012


### Duplicates

In [178]:
#checking if there is any duplicated designation in both datasets
wine_review1_df.duplicated().sum(), wine_review2_df.duplicated().sum()

(0, 0)

In [179]:
# Create an empty dictionary to store the count of duplicates for each column
duplicates_count = {}
columns_to_check=['designation','title']

# Iterate through each column and count duplicates
for col in columns_to_check:
    duplicates_count[col] = wine_review1_df[col].duplicated().sum()

# Display the count of duplicates for each column
for col, count in duplicates_count.items():
    print(f"Column '{col}' has {count} duplicates.")

Column 'designation' has 91991 duplicates.
Column 'title' has 11131 duplicates.


In [180]:
# Checking the duplicates in the column designation

wine_des_dup=wine_review1_df[wine_review1_df['designation'].duplicated()]
wine_des_dup[wine_des_dup["designation"].isna()]

,Unnamed: 0,country,description,designation,points,price,province,region_1,region_2,title,variety,winery
7,7,France,This dry and restrained wine offers spice in p...,NaN,87,24.0,Alsace,Alsace,NaN,Trimbach 2012 Gewurztraminer (Alsace),Gewürztraminer,Trimbach
11,11,France,"This is a dry wine, very spicy, with a tight, ...",NaN,87,30.0,Alsace,Alsace,NaN,Leon Beyer 2012 Gewurztraminer (Alsace),Gewürztraminer,Leon Beyer
12,12,US,"Slightly reduced, this wine offers a chalky, t...",NaN,87,34.0,California,Alexander Valley,Sonoma,Louis M. Martini 2012 Cabernet Sauvignon (Alex...,Cabernet Sauvignon,Louis M. Martini
14,14,US,Building on 150 years and six generations of w...,NaN,87,12.0,California,Central Coast,Central Coast,Mirassou 2012 Chardonnay (Central Coast),Chardonnay,Mirassou
19,19,US,"Red fruit aromas pervade on the nose, with cig...",NaN,87,32.0,Virginia,Virginia,NaN,Quiévremont 2012 Meritage (Virginia),Meritage,Quiévremont
...,...,...,...,...,...,...,...,...,...,...,...,...
129952,129952,US,This Zinfandel from the eastern section of Nap...,NaN,90,22.0,California,Chiles Valley,Napa,Houdini 2011 Zinfandel (Chiles Valley),Zinfandel,Houdini
129955,129955,New Zealand,"Like Dog Point's 2011 Chardonnay, this wine is...",NaN,90,40.0,Marlborough,NaN,NaN,Dog Point 2012 Chardonnay (Marlborough),Chardonnay,Dog Point
129961,129961,Italy,"Intense aromas of wild cherry, baking spice, t...",NaN,90,30.0,Sicily & Sardinia,Sicilia,NaN,COS 2013 Frappato (Sicilia),Frappato,COS
129967,129967,US,Citation is given as much as a decade of bottl...,NaN,90,75.0,Oregon,Oregon,Oregon Other,Citation 2004 Pinot Noir (Oregon),Pinot Noir,Citation


### Missing data 

In [181]:
#Checking the missing values in wine_review1_df 
wine_review1_df.isna().sum()

Unnamed: 0         0
country           63
description        0
designation    37465
points             0
price           8996
province          63
region_1       21247
region_2       79460
title              0
variety            1
winery             0
dtype: int64

#### Variety and designation missing data cleaning : 

In [182]:
#Cleaning the missing data

#Variety missing data 
wine_review1_df[wine_review1_df['variety'].isna()]
#Only one record is missing and most of the data for this wine is missing - Decided to remove the record
wine_review1_df = wine_review1_df.dropna(subset=['variety'])

#As title contains already the information of designation column - decision : drop designation 
wine_review1_df=wine_review1_df.drop(columns=['designation'])


#### Region missing data cleaning
Creating a unique new column 'region' retrieving the information from either region1 or region2 As 'region1' and 'region2' provide and contain the same information

In [ ]:
wine_review1_df
if wine_review1_df['region_1'].isna():
    wine_review1_df['region']=wine_review1_df['region_2']
else: 
    

## Concatenate both datasets